# Problem Statement

<img src="IMG_1440.jpg" width="700">

## Getting coordinates

<img src="A.jpg" width="500"> 

## Converting to a network

<img src="B.jpg" width="500">

In [1]:
import numpy as np

In [2]:
np.random.seed(0)

In [3]:
def generate_order(sku_list, order_lines, qty_range, qty_probability):
    order_lines = np.random.randint(1, order_lines+1)
    ordered_items = np.random.choice(sku_list, order_lines, replace=False)
    ordered_qty = [np.random.choice(qty_range, 1, p= qty_probability)[0] for _ in range(len(ordered_items))]
    order_list = dict(zip(ordered_items, ordered_qty))
    return order_list

In [4]:
def compute_rectilinear_distance(loc_A, loc_B):
    x_A, y_A = loc_A
    x_B, y_B = loc_B 
    return np.abs(x_A - x_B) + np.abs(y_A - y_B)

In [5]:
def compute_distance_traveled(order_list, distance_dict):
    lst = list(order_list.keys())
    first_loc = lst[0]
    last_loc = lst[-1]

    lst_chunk_of_2 = []
    for i in range(len(lst)-1):
        lst_chunk_of_2.append([lst[i], lst[i+1]])

    lst_chunk_of_2.insert(0, ['Depot', first_loc])
    lst_chunk_of_2.insert(len(lst_chunk_of_2), [last_loc, 'Depot'])

    distance_traveled_list = []
    for locs in lst_chunk_of_2:
        [skuA, skuB] = locs
        loc_skuA = distance_dict[skuA]
        loc_skuB = distance_dict[skuB]
        skuA_to_skuB_distance = compute_rectilinear_distance(loc_skuA, loc_skuB)
        distance_traveled_list.append(skuA_to_skuB_distance)

    distance_traveled = np.sum(np.array(distance_traveled_list))
    return distance_traveled

In [6]:
def picker_tour(order_list, distance_dict, pick_time, travel_speed):
    depot = distance_dict['Depot']
    if len(order_list)==1:
        sku = list(order_list.keys())[0]
        loc_sku = distance_dict[sku]
        items_picked = list(order_list.values())[0]
        distance_traveled = 2*compute_rectilinear_distance(depot, loc_sku)
        
    else:
        items_picked = np.sum(list(order_list.values()))
        distance_traveled = compute_distance_traveled(order_list, distance_dict)
    
    tour_time = items_picked*pick_time + (distance_traveled/travel_speed)   
    productivity = items_picked*3600/tour_time
    return distance_traveled, items_picked, tour_time, productivity

### Data

In [7]:
distance_dict = {'Depot': (12, 0), 'A': (4, 10), 'B': (4, 6), 'C': (8, 10), 
                 'D': (8, 6), 'E': (16, 10), 'F': (16, 6), 'G': (20, 10), 'H': (20, 6)}

In [8]:
sku_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
order_lines = 2
qty_range = [1, 2, 3, 4]
qty_probability = [0.2, 0.3, 0.3, 0.2]
pick_time = 1 # sec
travel_speed = 3 # ft/sec

In [9]:
no_of_orders = 10000

## Solution

In [10]:
distance_traveled_tracker = []
items_picked_tracker = []
tour_time_tracker = []
productivity_tracker = []
for _ in range(no_of_orders):
    order_list = generate_order(sku_list, order_lines, qty_range, qty_probability)
    distance_traveled, items_picked, tour_time, productivity = picker_tour(order_list, distance_dict, pick_time, travel_speed)
    distance_traveled_tracker.append(distance_traveled)
    items_picked_tracker.append(items_picked)
    tour_time_tracker.append(tour_time)
    productivity_tracker.append(productivity)

avg_distance_traveled = np.mean(np.array(distance_traveled_tracker))
avg_items_picked = np.mean(np.array(items_picked_tracker))
avg_tour_time = np.mean(np.array(tour_time_tracker))
avg_productivity = np.mean(np.array(productivity_tracker))

#print('Average distance traveled {:0.2f} ft'.format(avg_distance_traveled))
#print('Average no. of items picked {:0.2f}'.format(avg_items_picked))
#print('Average picking tour time {:0.2f} sec'.format(avg_tour_time))
print('Average productivity {:0.2f} items/hour'.format(avg_productivity))

Average productivity 883.04 items/hour
